## Imports

In [ ]:
#!pip install imbalanced-learn

In [ ]:

########################### (https://github.com/curiousily/TensorFlow-on-Android-for-Human-Activity-Recognition-with-LSTMs/blob/master/human_activity_recognition.ipynb) imports
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt


from scipy import stats
import tensorflow as tf
import seaborn as sns
from pylab import rcParams
from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

%matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42


from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel

from sklearn.utils import shuffle

from matplotlib.pyplot import figure

from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier

from sklearn.decomposition import PCA

# Upload Data 

---------------------------------



Train

In [ ]:
from google.colab import files
uploaded_signal_train = files.upload()

Test

In [ ]:
from google.colab import files
uploaded_signal_test = files.upload()

### Reading in segments & labels

Train

In [ ]:
# getting keys, which is file names of npy 
list_of_dataframes_train = [key for key in uploaded_signal_train.keys()]


# set up list to hold all loaded npy 
all_dataframe_train = [] 


for i in range(len(list_of_dataframes_train)):

    # load in the data 
    dataframe_train = pd.read_csv(list_of_dataframes_train[i])


    # append the data to 'all' list
    all_dataframe_train.append(dataframe_train) 


all_df_train = pd.concat(all_dataframe_train)

Test

In [ ]:
# getting keys, which is file names of npy 
list_of_dataframes_test = [key for key in uploaded_signal_test.keys()]


# set up list to hold all loaded npy 
all_dataframe_test = [] 


for i in range(len(list_of_dataframes_test)):

    # load in the data 
    dataframe_test = pd.read_csv(list_of_dataframes_test[i])


    # append the data to 'all' list
    all_dataframe_test.append(dataframe_test) 


all_df_test = pd.concat(all_dataframe_test)


# Quick Look 

In [ ]:
all_df_train.head()

In [ ]:
all_df_test.head()

# Train Test Split 

In [ ]:
# Getting X_train & y_train'
X_train = all_df_train.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run', 'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands'	], axis = 1)
y_train = all_df_train['Label_segment'].values

In [ ]:
# Getting X_train & y_train
X_test = all_df_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run',  'X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands'], axis = 1)
y_test = all_df_test['Label_segment'].values

---------------------------

Standard Scale 

In [ ]:
# Set up ss for non-shuffled data
ss = StandardScaler()


# fit train & transform test
X_train_scale = ss.fit_transform(X_train)
X_test_scale = ss.transform(X_test)

-----------------------------------------

In [ ]:
from sklearn.decomposition import PCA

# PCA set for 95% of variance 
pca = PCA(n_components=0.95)

pca.fit(X_train_scale)

In [ ]:
# PCA set for 95% of variance  

X_train_scale_pca = pca.transform(X_train_scale)
X_test_scale_pca = pca.transform(X_test_scale)

# quick print of shapes to see difference
print(f'Train data:\nBefore pca shape: {X_train_scale.shape} , after pca applied: {X_train_scale_pca.shape} ')
print(f'\nTest data:\nBefore pca shape: {X_test_scale.shape} , after pca applied: {X_test_scale_pca.shape} ')



Check if SMOTE helps

In [ ]:
print(Counter(y_train))

In [ ]:
# transform the dataset
oversample = SMOTE()
X_train_scale_smote, y_train_smote = oversample.fit_resample(X_train_scale_pca, y_train)

In [ ]:
print(Counter(y_train_smote))

In [ ]:
# quick print of shapes to see difference
print(f'Train data:\nBefore pca shape: {X_train_scale.shape} , after pca applied: {X_train_scale_smote.shape} ')


----------------------------------------------

# PCA - No SMOTE 

Check Some Classifiers 

In [ ]:

import time
from sklearn.metrics import accuracy_score, log_loss , recall_score , f1_score, precision_score , roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import PassiveAggressiveClassifier , LogisticRegression
from sklearn.semi_supervised import LabelPropagation
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.linear_model import LogisticRegression

classifiers = [
               
    # Too much computation to run now since using entire data and not segmenting on location 
    ##KNeighborsClassifier(),
    ##GradientBoostingClassifier(),
    ##LabelPropagation(),
    # DecisionTreeClassifier(),
    # RandomForestClassifier(),
    # AdaBoostClassifier(),
    # GaussianNB(),
    # LinearDiscriminantAnalysis(),
    # QuadraticDiscriminantAnalysis(),
    # LinearSVC(),
    # SGDClassifier(),
    # MLPClassifier(),
    # PassiveAggressiveClassifier(),
    # ExtraTreesClassifier(),
    # BaggingClassifier(),

    DecisionTreeClassifier(criterion = 'entropy' , max_depth = 10 , min_samples_leaf = 8 , min_samples_split = 3  ),
    MLPClassifier(hidden_layer_sizes = (50, 20) , activation = 'tanh' , solver = 'adam' , alpha = 0.01 , learning_rate = 'adaptive' ),
    LogisticRegression(C = 1 , multi_class = 'multinomial' , penalty = 'l2' , solver = 'newton-cg' )


  
    
     ]


# list to hold for dataframe
classifier_name_list = []
train_acc_list = []
train_bacc_list = []
test_acc_list = []
test_bacc_list = []
train_recall_list = []
test_recall_list = []
train_precision_list = []
test_precision_list = []
train_f1_list = []
test_f1_list = []
training_timing = [] 
training_pred_timing = [] 
testing_pred_timing = [] 


for clf in classifiers:

    name = clf.__class__.__name__
    
    print("="*50)
    print(name)

    # just for timing model
    training_time0 = time.time()


    ################### This changes across runs ####################################################
    clf.fit(X_train_scale_pca, y_train)

    # finished training 
    training_time1 = time.time()

    training_time = training_time1 - training_time0 
    
    print('****Results****')

    # just for timing model
    test_time0 = time.time()
    # Test Predictions

    ################### This changes across runs ####################################################
    test_predictions = clf.predict(X_test_scale_pca)
    # just for timing model
    test_time1 = time.time()

    test_time = test_time1 - test_time0


    # Test Metrics
    acc            = accuracy_score(y_test, test_predictions)
    bal_acc        = balanced_accuracy_score(y_test, test_predictions)
    recall_test    = recall_score(y_test, test_predictions, average = 'weighted')
    f1_test        = f1_score(y_test, test_predictions ,  average = 'weighted')
    precision_test = precision_score(y_test, test_predictions,  average = 'weighted') 


    # just for timing model
    training_p_time0 = time.time()

    # Train Predictions

    ################### This changes across runs ####################################################
    train_predictions = clf.predict(X_train_scale_pca)

    # just for timing model
    training_p_time1 = time.time()

    training_p_time = training_p_time1 - training_p_time0

    # Train Metrics

    ################### This changes across runs ####################################################
    train_acc       = accuracy_score(y_train, train_predictions)
    train_bal_acc   = balanced_accuracy_score(y_train, train_predictions)
    recall_train    = recall_score(y_train, train_predictions , average = 'weighted')
    f1_train        = f1_score(y_train, train_predictions ,  average = 'weighted')
    precision_train = precision_score(y_train, train_predictions,  average = 'weighted') 

    print("\n\nTest Classification Report\n")
    print(classification_report(y_test, test_predictions))


    # append to list to make a dataframe 
    classifier_name_list.append(name)
    
    training_timing.append(training_time)
    training_pred_timing.append(training_p_time)
    testing_pred_timing.append(test_time)


    train_acc_list.append(train_acc)
    test_acc_list.append(acc)

    train_bacc_list.append(train_bal_acc)
    test_bacc_list.append(bal_acc)

    train_recall_list.append(recall_train)
    test_recall_list.append(recall_test)

    train_precision_list.append(precision_train)
    test_precision_list.append(precision_test)

    train_f1_list.append(f1_train)
    test_f1_list.append(f1_test)




print("="*50)




In [ ]:
train_metrics_df = pd.DataFrame()
test_metrics_df = pd.DataFrame()


train_metrics_df['Classifier'] = classifier_name_list
test_metrics_df['Classifier'] = classifier_name_list


# F1 Score First 
train_metrics_df['Train F1'] = train_f1_list
test_metrics_df['Test F1'] = test_f1_list 

# Recall
train_metrics_df['Train Recall'] = train_recall_list
test_metrics_df['Test Recall'] = test_recall_list

# Precision 
train_metrics_df['Train Precision'] = train_precision_list
test_metrics_df['Test Precision'] = test_precision_list

# Bal Acc
train_metrics_df['Train Balanced Accuracy'] = train_bacc_list
test_metrics_df['Test Balanced Accuracy'] = test_bacc_list

# Accuracy 
train_metrics_df['Train Accuracy'] = train_acc_list
test_metrics_df['Test Accuracy'] = test_acc_list 


#train_metrics_df.sort_values("Train F1" , ascending=False , inplace=True)
train_metrics_df.set_index('Classifier' , inplace=True)

#test_metrics_df.sort_values("Test F1" , ascending=False , inplace=True)
test_metrics_df.set_index('Classifier' , inplace=True)

train_metrics_df['Model Training Taken (seconds)']  = training_timing
test_metrics_df['Model Training Taken (seconds)']  = training_timing

train_metrics_df['Model Prediction Time Taken (seconds)']  = training_pred_timing
test_metrics_df['Model Prediction Time Taken (seconds)']  = testing_pred_timing




# display df
display(train_metrics_df)

In [ ]:
# display df
display(test_metrics_df)

---------------------------------------

SVM 

In [ ]:
from sklearn.svm import SVC

In [ ]:
svc_clf = SVC(C = 0.3)


svc_clf.fit(X_train_scale_pca , y_train)

# Predict Train
preds_svc = svc_clf.predict(X_train_scale_pca)

# Classification report 
print("Train Data")
print(classification_report(y_train,preds_svc))

In [ ]:
# Predict Test
preds_svc = svc_clf.predict(X_test_scale_pca)

print("Test Data")
# Classification report 
print(classification_report(y_test,preds_svc))

---------------------------------------------

In [ ]:
svc_clf_smote = SVC(C = 0.3)


svc_clf.fit(X_train_scale_smote , y_train_smote)

# Predict Train
preds_svc = svc_clf.predict(X_train_scale_smote)

# Classification report 
print("Train Data")
print(classification_report(y_train , preds_svc))

In [ ]:
def holdout_checker_svc(smote = False ):

  # Read in Fresh Data to see what predictions looks like
  from google.colab import files
  uploaded_test = files.upload()

  # getting keys, which is file names of csv
  val_file_name = [key for key in uploaded_test.keys()]

  # read in csv 
  signal_test = pd.read_csv(val_file_name[0])


  # Getting X_train & y_train
  X_data = signal_test.drop(['Unnamed: 0' , 'Label_segment' , 'Participant_ID' , 'Participant_Run','X_Acc_Move_FFT_EnergyBands', 	'Y_Acc_Move_FFT_EnergyBands', 	'Z_Acc_Move_FFT_EnergyBands' ], axis = 1)
  y_data = signal_test['Label_segment'].values

  # scale with already fit ss 
  X_data_scale = ss.transform(X_data)

  # feature selection
  X_data_scale_pca = pca.transform(X_data_scale)



  # if selecting smote trained model
  if smote == True:
    hold_preds = model_smote.predict(X_data_scale_fs)

  # else non-smote trained model
  else:
    hold_preds = svc_clf.predict(X_data_scale_pca)

  # take max of predictions 
  #max_predictions = np.argmax(hold_preds, axis=1)

  # metrics 
  print("\n---------------------- Metrics ----------------------------------------")

  print("Accuracy : \t\t" ,accuracy_score(y_data, hold_preds))
  print("Balanced Accuracy : \t" , balanced_accuracy_score(y_data, hold_preds))
  #print("F1 Score : \t\t" , f1_score(y_data, hold_preds, average='weighted'))


  # set up labels 
  LABELS = ['Go', 'Turn1',  'Turn2' , 'Walk1', 'Walk2', 'Sit']

  # classification report 
  print("\n------------------- HoldOut Classification Report ---------------")
  print(classification_report(y_data , hold_preds))
  print(" ")

  # confusion matrix
  confusion_matrix_out = metrics.confusion_matrix(y_data, hold_preds )

  plt.figure(figsize=(14, 10))
  sns.heatmap(confusion_matrix_out, xticklabels=LABELS, yticklabels=LABELS, annot=True ,fmt="d" );
  plt.title("HoldOut Data Confusion matrix")
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

  # lastly just printing actual predictions
  print("\n0 = Go \t\t 1 = Turn 1 \t 2 = Turn 2")
  print("\n3 = Walk 1 \t 4 = Walk 2 \t 5 = Sit")

  print(" ")
  print(hold_preds)

  print(Counter(hold_preds))

In [ ]:
holdout_checker_knn()